# Embeddings Neuronales


En este notebook, se utiliza la librería [fastText](https://en.wikipedia.org/wiki/fastText) como modelo de generación de embeddings de palabras, luego se entrena un modelo k-means para obtener clusters, y se marca cada comentario con un determinado número de cluster.

### Importanción de librería requeridas

In [1]:
import os
import pickle

import warnings
import pickle
from collections import Counter

import numpy as np
import pandas as pd

import gensim.corpora as corpora
from gensim.models import FastText

from sklearn.metrics import pairwise_distances

from clustering_utils import vectorize, mbkmeans_clusters

warnings.filterwarnings("ignore", category=DeprecationWarning)
np.random.seed(42)

%matplotlib inline


### Definición de variables globales

In [2]:
TEXT_FILE_READ = 'docs/preprocessing_reddit_data.csv'
TEXT_SAVE_FILE = 'docs/reddit_data_fasttext.csv'
PICKLE_KMEANS = 'docs/models/fasttext_kmeans.model'
SAVE_TEST_CSV=False

n_clusters = 120

### Lectura de los comentarios de Reddit

Los comentarios fueron previamente preprocesados (Ver en TODO).

In [3]:
df = pd.read_csv(TEXT_FILE_READ)
df['lemma_tokens'] = df['lemma_tokens'].apply(eval)

### Vocabulario

In [4]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemma_tokens'])

# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)

# Creating a corpus object
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

processed_corpus = df['lemma_tokens']


### Entrenamiento del modelo FastText

In [5]:
model = FastText(sentences=processed_corpus, vector_size=100, window=5, min_count=1, workers=1)
model.train(processed_corpus, total_examples=len(processed_corpus), epochs=100)
model.save("docs/models/fasttext.model")

### Generación de vectores desde documentos

In [13]:

vectorized_docs = vectorize(processed_corpus, model=model)
len(vectorized_docs), len(vectorized_docs[0])



(27791, 100)

### Generación de clusters

In [14]:
clustering, cluster_labels = mbkmeans_clusters(
    X=vectorized_docs,
    k=n_clusters,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": df["body"].values,
    "tokens": [" ".join(text) for text in processed_corpus],
    "cluster": cluster_labels
})

with open(PICKLE_KMEANS, 'wb') as f:
    pickle.dump(clustering, f)

For n_clusters = 120
Silhouette coefficient: 0.01
Inertia:1525484.9615074536
Silhouette values:
    Cluster 107: Size:10 | Avg:0.26 | Min:-0.04 | Max: 0.39
    Cluster 43: Size:20 | Avg:0.15 | Min:0.02 | Max: 0.33
    Cluster 34: Size:216 | Avg:0.15 | Min:-0.03 | Max: 0.30
    Cluster 54: Size:145 | Avg:0.13 | Min:-0.08 | Max: 0.31
    Cluster 113: Size:159 | Avg:0.11 | Min:-0.08 | Max: 0.31
    Cluster 30: Size:91 | Avg:0.11 | Min:-0.04 | Max: 0.25
    Cluster 68: Size:69 | Avg:0.11 | Min:-0.05 | Max: 0.29
    Cluster 11: Size:85 | Avg:0.11 | Min:-0.01 | Max: 0.22
    Cluster 111: Size:35 | Avg:0.10 | Min:0.01 | Max: 0.26
    Cluster 112: Size:54 | Avg:0.10 | Min:-0.03 | Max: 0.26
    Cluster 27: Size:48 | Avg:0.09 | Min:-0.17 | Max: 0.30
    Cluster 39: Size:328 | Avg:0.09 | Min:-0.18 | Max: 0.24
    Cluster 89: Size:68 | Avg:0.09 | Min:-0.06 | Max: 0.30
    Cluster 17: Size:18 | Avg:0.08 | Min:-0.02 | Max: 0.21
    Cluster 101: Size:160 | Avg:0.08 | Min:-0.09 | Max: 0.31
    Cluster

### *Top terms* por cluster (basado en los centroides de los clusters)

In [15]:
print("Most representative terms per cluster (based on centroids):")
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=10)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")



Most representative terms per cluster (based on centroids):
Cluster 0: macri macri~~. macrium -macri \^macri macris mracri macri-peño mauriciomacri macr 
Cluster 1: acá acábo ac acv acaa acné aca \*sacá acabé acabás 
Cluster 2: ganar votar votastar voto votarar ganarir votos perderiar idiotar votarl 
Cluster 3: tener tenerla \-tener obtener abstener detener tenembaum retener sostener tenenbaum 
Cluster 4: calientenla calor caliente calo calentón calchaqui calienta escalofrío caliento agua 
Cluster 5: leer leeré leerlo leerme galeer leet lee leen libreer lees 
Cluster 6: fiacar bacar \-car orcar acar petacar facar cacar sacar evalúar 
Cluster 7: laempresanoesresponsablepordesgraciaspersonalnisupersonalestargenuinamenteinteresadoenloqueocurraenlavidadelcliente conformidad conectividad integridad invecilidad coeficiente intimidad descuidadamente entidad inconstitucionalidad 
Cluster 8: compralar comprendés compro compren comprendido compra comprar comprale comprate compremosle 
Cluster 9:

### *Top terms* por cluster (basado en las palabras más frecuentes)

In [16]:
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_frequent = Counter(" ".join(df_clusters.query(f"cluster == {i}")["tokens"]).split()).most_common(5)
    for t in most_frequent:
        tokens_per_cluster += f"{t[0]}({str(t[1])}) "
    print(f"Cluster {i}: {tokens_per_cluster}")


Cluster 0: macri(54) ah(7) macrista(4) /s(4) medio(3) 
Cluster 1: acá(148) aca(10) argentina(7) ver(7) sub(5) 
Cluster 2: ganar(139) voto(103) votar(73) perder(53) elección(27) 
Cluster 3: tener(160) él(11) año(11) tenés(10) tenia(7) 
Cluster 4: calor(83) agua(63) aire(32) team(30) frio(29) 
Cluster 5: leer(164) libro(21) comentario(17) escribir(11) él(11) 
Cluster 6: sacar(129) car(39) buscar(20) banco(14) gente(13) 
Cluster 7: gente(59) realidad(46) idea(38) cantidad(30) tenés(29) 
Cluster 8: comprar(148) compra(53) vender(38) compro(26) barato(25) 
Cluster 9: negro(57) chino(26) tiro(22) raro(16) puro(15) 
Cluster 10: vo(138) decir(11) favor(10) gracias(6) so(5) 
Cluster 11: the(123) of(30) is(11) vs(7) for(6) 
Cluster 12: elección(15) nación(13) inflación(12) acción(11) opción(8) 
Cluster 13: contar(34) matar(23) costar(17) faltar(15) control(11) 
Cluster 14: pagar(174) impuesto(66) cobrar(26) plan(23) pago(20) 
Cluster 15: mano(96) man(10) duro(5) humano(4) querer(4) 
Cluster 16: 

### Recupere los documentos más representativos (basados en los centroides de los clústeres) para un cluster en particular

In [17]:
test_cluster = 0
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:10]:
    print( df["body"].values[d])
    print("-------------")

Macri!! Shot. Macri!! Shot. Macr burp. *vomita*
-------------
Macri: uff le moví el cerebro
-------------
Si Macri moviliza Micros, siendo 400, no los podemos llamar Macros?. Macross Delta 400 Macri no tensei
-------------
Macri parece Bowie en la portada de *Heathen*:. https://en.wikipedia.org/wiki/Heathen\_(David\_Bowie\_album)
-------------
La única propuesta de Tolosa Paz es "ah pero Macri." Literalmente. Tiene que dejar de mirar C5N. A la gente que no los votó les importa poco Macri y lo que haya o no hecho.
-------------
Ahora que me acuerdo Macri también era el HG. También lo reivindican a Néstor con ese título.
-------------
No... Nosotros hicimos todo bien, son los votantes que están engañados por los medios hegemónicos, Macri y el FMI, eso debe ser ...
-------------
Alguien viene contando cuantos macris dijo esta sra horrenda? Escuche 95% macri 6% propuestas 1% margen d error
-------------
Y si perdemos la culpa será de Macri, Larreta, Milei, Los gorilas, los medios hegemónic

In [18]:
test_v = vectorize([['defender', 'peso', 'siente', 'corazón', 'compro', 'pesos', 'tasa', 'fijo', 'año']], model=model)
prediction = clustering.predict(test_v)
print(prediction)

[82]


In [12]:
def get_cluster(row):
    test_v = vectorize([str(row).split(" ")], model=model)
    return clustering.predict(test_v)

df['cluster'] = df.apply(lambda row: get_cluster(row['body_preprocessing']) , axis = 1)

In [13]:
# Show
df.head(10)

,score,id,flair,comms_num,body,comment_parent_id,is_replay,lemma_tokens,body_preprocessing,body_no_lemmas,cluster
0,1,hfw14mt,Discusion🧐,1,todo para decir que tapaste el baño. tira un b...,q44kw3,False,"[tapastar, baño, tirar, balde, agua]",tapastar baño tirar balde agua,tapaste baño. tira balde agua pa,[112]
1,1,hfw41eh,Discusion🧐,0,"sopapa primero master, si hay tapón te vas a t...",hfw14mt,True,"[sopapa, master, tapón, teñir, medias]",sopapa master tapón teñir medias,"sopapa master, tapón vas a teñir medias!",[89]
2,1,hfw1ao2,Discusion🧐,0,"Usas la sopapa, o tiras agua caliente con un b...",q44kw3,False,"[sopapa, tira, agua, caliente, balde]",sopapa tira agua caliente balde,"sopapa, o tiras agua caliente balde",[112]
3,1,hfw3jof,Discusion🧐,2,Lo que he probado que siempre me dio resultado...,q44kw3,False,"[probado, resultado, sellar, boca, inodoro, ti...",probado resultado sellar boca inodoro tirar ca...,probado resultado sellar boca inodoro y tirar ...,[51]
4,1,hfw6v4i,Discusion🧐,0,Estas cobrando por dar mantenimiento y no sabe...,q44kw3,False,"[cobrar, mantenimiento, carajo, kjjjjjjjjj, vi...",cobrar mantenimiento carajo kjjjjjjjjj viviria...,cobrando mantenimiento y carajo? kjjjjjjjjj. v...,[5]
5,1,hfw26iv,Discusion🧐,0,"Si tenes algo con punta, metelo y hace un poco...",q44kw3,False,"[tener, punta, metelo, fuerza, romper, tapa, b...",tener punta metelo fuerza romper tapa baño tir...,"tenes punta, metelo y fuerza, ""romper"" tapa ba...",[112]
6,1,hfw2gof,Discusion🧐,1,"Con una manguera para regar el jardín, si tene...",q44kw3,False,"[manguerar, regar, jardín, tener, probar]",manguerar regar jardín tener probar,"manguera regar jardín, tenes, podes probar.",[107]
7,1,hfw5s13,Discusion🧐,0,"despues regas el jardin y se lava sola, solo q...",hfw2gof,True,"[rega, jardin, lavar, tenés, lavar, tú, mano, ...",rega jardin lavar tenés lavar tú mano pulgar c...,"regas jardin y lava sola, tenés lavarte manos ...",[91]
8,1,hfw3air,Discusion🧐,0,La respuesta real es que se venden unos caños ...,q44kw3,False,"[respuesta, real, vender, caño, alambrado, dec...",respuesta real vender caño alambrado decir cañ...,"respuesta real venden caños “alambrados”, diga...",[97]
9,7,hfvxa6w,Discusion🧐,3,Mi alfajor favorito es el Havana,q443eo,False,"[alfajor, favorito, havana]",alfajor favorito havana,alfajor favorito havana,[33]


In [21]:
df.to_csv(TEXT_SAVE_FILE, index=False)

In [22]:
TEST_CLUSTER_PATH = 'docs/test/fasttext_comments_per_cluster/'

if SAVE_TEST_CSV:
    os.makedirs(TEST_CLUSTER_PATH,exist_ok=True)

    for i in range(n_clusters):
        df[(df["cluster"] == i)][['flair', 'body']].to_csv(TEST_CLUSTER_PATH + str(i) + '.csv')
